<h3>>> Create sqlite db</h3>

In [ ]:
mlflow ui --backend-store-uri sqlite:///mlflow.db

<h3>>> Connect to db</h3>

In [ ]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')

<h3>>> Set Experiment</h3>

In [ ]:
mlflow.set_experiment('my-new-experiment')   # if Doesn't exist it will create this experiment

<h3>>> run mlflow</h3>

In [ ]:
with mlflow.start_run():
    
    # Log information about the model run
    mlflow.set_tag("developer", "Seyedmahdi Amin")

    # Any extra information have an effect on our model performance
    mlflow.log_param("train-data-path", "./data/train.csv")
    mlflow.log_param("valid-data-path", "./data/valid.csv")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    # train model here
    # predict validation model and calculate error here 
    rmse = 3.231  # For example I calculate rmse here
    # Any metric data
    mlflow.log_metric("rmse", rmse)

<h3>>> mlflow autolog</h3>
<a>https://www.mlflow.org/docs/latest/tracking.html#automatic-logging</a>

In [ ]:
# We use autulog() to save all parameters of a model
# check this link ==> https://www.mlflow.org/docs/latest/tracking.html#automatic-logging
# Call mlflow.autolog() before your training code.
# This will enable autologging for each supported library you have installed as soon as you import it.

import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.autolog()
db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators = 100, max_depth = 6, max_features = 3)
rf.fit(X_train, y_train)

<h3>>> Save the model</h3>

In [ ]:
# First way
# When we train a model and save all the things that weee need in a folder 
# after that we can write that folder name to log all the files in that to log_artifact 
mlflow.log_artifact(local_path="myFolder/myfile.txt", artifact_path="models/")

# second way
# when we want to save all the things in a foder
mlflow.sklearn.log_model(rf ,artifact_path="models/") # "rf" is the name of our model


<h3>>> Model Registry</h3>

In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
# List all Experiments by python
client.list_experiments()

# Create new Experiment
client.create_experiment(name='ali')

# list all registered model names
client.list_registered_models()

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 2000',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")


In [ ]:
# Register an experiment with id 277d1e18e6c848c9821bf64a7892eeb2 
run_id = "277d1e18e6c848c9821bf64a7892eeb2"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name="Diabets")

In [ ]:
# To get latest versions
model_name = "Diabets"
latest_versions = client.get_latest_versions(name = model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")


In [ ]:
# To change the stage
model_Version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name = model_name,
    version = model_Version,
    stage = new_stage,
    archive_existing_versions = False
)

In [ ]:
from datetime import datetime
date = datetime.today().date()
model_Version = 2
new_stage = "Staging"

client.update_model_version(
    name = model_name,
    version = model_Version,
    description= f"The model version {model_Version} was transitioned to {new_stage} on {date}"
)